
# Using SQL Agents with LangChain

This notebook demonstrates how to use LangChain's SQL Agent to query an SQL database. We'll use a mock banking dataset stored in SQLite to answer questions about customers, interest rates, and more.

### Objectives:
- Understand how to connect LangChain's SQL Agent to a database.
- Learn to run natural language queries on the database.
- Extract insights from database tables using an LLM-powered agent.

Let's begin!



## Step 1: Import Libraries

We'll start by importing the necessary libraries for working with LangChain's SQL Agent. These include:
- `langchain.agents` for creating the SQL agent.
- `sqlite3` for database interaction.
- `langchain.chat_models` for utilizing OpenAI's chat models.
- `langchain.sql_database` for handling SQL databases.


In [1]:
from langchain.agents import create_sql_agent
import sqlite3
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase


## Step 2: Set Up the Database

Here, we define the path to our SQLite database and initialize the `SQLDatabase` object, which allows LangChain to interact with the database.


In [2]:

db_path = "mock_bank_data.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")



## Step 3: Initialize the Language Model

We'll use OpenAI's GPT-3.5-turbo model with a temperature of 0.0 to ensure deterministic responses. This model powers the SQL agent.


In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

C:\Users\nande\AppData\Local\Temp\ipykernel_1136\2335869711.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)



## Step 4: Create the SQL Agent

Now, we create the SQL agent by combining the LLM and the database. The agent is responsible for translating natural language queries into SQL commands and executing them.


In [4]:
sql_agent = create_sql_agent(
        llm=llm,
        db=db,
        agent_type="openai-tools",
        verbose=True,
    )


## Step 5: Query the Database

Here are some examples of queries you can run using the SQL agent:
1. How many customers are there in the database?
2. What is the average interest rate by product type?
3. What was the first payment default average in monetary values for overdue days > 30 in 2024?

The agent automatically handles the SQL generation and execution for these queries.


In [5]:
n_customers = sql_agent.run("How many customers are there?")
print(n_customers)

C:\Users\nande\AppData\Local\Temp\ipykernel_1136\2828441748.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  n_customers = sql_agent.run("How many customers are there?")




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Accounts, Cost_of_Capital, Customers, Installments, Products, Transactions
Invoking: `sql_db_schema` with `{'table_names': 'Customers'}`



CREATE TABLE "Customers" (
	"Customer_ID" INTEGER, 
	"Name" TEXT NOT NULL, 
	"Date_of_Birth" DATE, 
	"Gender" TEXT, 
	"Address" TEXT, 
	"Phone_Number" TEXT, 
	"Email" TEXT, 
	"Account_Open_Date" DATE, 
	"Relationship_Status" TEXT, 
	"Risk_Score" INTEGER, 
	PRIMARY KEY ("Customer_ID")
)

/*
3 rows from Customers table:
Customer_ID	Name	Date_of_Birth	Gender	Address	Phone_Number	Email	Account_Open_Date	Relationship_Status	Risk_Score
1	Stacey Lowe	1966-12-21	Male	1158 Flores Plain Suite 055
Kristinachester, MT 63422	(599)677-4230	kristopher35@example.org	2018-02-21	Inactive	1
2	Jodi Mclean	1973-07-15	Male	5906 Thomas Meadows Suite 267
Obrienfurt, WV 80928	330-907-9590	allendavid@example.org	2020-02-23	Inactive	83
3	David Rangel	1973-10-09	Male	13755 Sawyer Field Ap

In [6]:
avg_interest = sql_agent.run("What is the average interest rate by product type?")
print(avg_interest)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Accounts, Cost_of_Capital, Customers, Installments, Products, Transactions
Invoking: `sql_db_schema` with `{'table_names': 'Products'}`



CREATE TABLE "Products" (
	"Product_ID" INTEGER, 
	"Product_Name" TEXT, 
	"Product_Type" TEXT, 
	"Interest_Rate" REAL, 
	"Fees" REAL, 
	"Launch_Date" DATE, 
	"End_Date" DATE, 
	PRIMARY KEY ("Product_ID")
)

/*
3 rows from Products table:
Product_ID	Product_Name	Product_Type	Interest_Rate	Fees	Launch_Date	End_Date
1	Cost Product	Loan	13.94	380.72	2020-06-15	2026-11-21
2	Interesting Product	Loan	7.5	280.45	2015-07-09	None
3	Audience Product	Savings Plan	2.31	292.28	2017-01-19	2025-07-17
*/
Invoking: `sql_db_query` with `{'query': 'SELECT Product_Type, AVG(Interest_Rate) AS Average_Interest_Rate FROM Products GROUP BY Product_Type'}`


[('Credit Card', 6.108888888888889), ('Loan', 9.148333333333333), ('Savings Plan', 6.944)]The average interest rates by product typ

In [7]:
fpd = sql_agent.run("What was the first payment default average in monetary values with overdue days>30 in 2024?") 
print(fpd)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Accounts, Cost_of_Capital, Customers, Installments, Products, Transactions
Invoking: `sql_db_schema` with `{'table_names': 'Transactions, Installments'}`



CREATE TABLE "Installments" (
	"Installment_ID" INTEGER, 
	"Account_ID" INTEGER, 
	"Product_ID" INTEGER, 
	"Installment_Start_Date" DATE, 
	"Installment_End_Date" DATE, 
	"Installment_Amount" REAL, 
	"Paid_Amount" REAL, 
	"Remaining_Balance" REAL, 
	PRIMARY KEY ("Installment_ID"), 
	FOREIGN KEY("Account_ID") REFERENCES "Accounts" ("Account_ID"), 
	FOREIGN KEY("Product_ID") REFERENCES "Products" ("Product_ID")
)

/*
3 rows from Installments table:
Installment_ID	Account_ID	Product_ID	Installment_Start_Date	Installment_End_Date	Installment_Amount	Paid_Amount	Remaining_Balance
1	34	Data Product	2024-02-07	2024-04-12	2015.24	890.96	1124.28
2	23	Interesting Product	2022-08-09	2025-05-31	3330.58	2398.66	931.9200000000001
3	79	Paper Product	2023-08-19


## Conclusion

This notebook demonstrated how to use LangChain's SQL Agent to interact with a database using natural language. Feel free to extend this by adding more complex queries or connecting to your own datasets!
